In [1]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
from sklearn.linear_model import LinearRegression
from scipy.stats import spearmanr

In [2]:
data_path = './data/'
df_X_train = pd.read_csv(data_path+'X_train.csv')#.drop(['COUNTRY'], axis=1).fillna(0) # given by the benchmark
df_X_test = pd.read_csv(data_path+'X_test.csv')
df_y_train = pd.read_csv(data_path+'Y_train.csv')

In [17]:
# French and Deutch sepration
df = pd.merge(df_X_train, df_y_train, on='ID')
df_X_train_fr = df.loc[df['COUNTRY'] == 'FR', df.columns[:-1]].drop(columns=['COUNTRY']).fillna(0)
df_y_train_fr = df.loc[df['COUNTRY'] == 'FR', ['ID', 'TARGET']]

df_X_train_de = df.loc[df['COUNTRY'] == 'DE', df.columns[:-1]].drop(columns=['COUNTRY']).fillna(0)
df_y_train_de = df.loc[df['COUNTRY'] == 'DE', ['ID', 'TARGET']]

df_X_test_fr = df_X_test.loc[df_X_test['COUNTRY'] == 'FR', df_X_test.columns].drop(columns=['COUNTRY']).fillna(0)
df_X_test_de = df_X_test.loc[df_X_test['COUNTRY'] == 'DE', df_X_test.columns].drop(columns=['COUNTRY']).fillna(0)


In [11]:
def show_spearman_correlation(y_predict, df_y_train):
    print(spearmanr(y_predict, df_y_train['TARGET']).correlation)

Linear regression

In [26]:
model = LinearRegression()
df_X_train.drop(['COUNTRY'], axis=1).fillna(0)
# df_y_train['TARGET']
model.fit(df_X_train.drop(['COUNTRY'], axis=1).fillna(0), df_y_train['TARGET'])
# model.predict(df_X_train_fr)
show_spearman_correlation(model.predict(df_X_train.drop(['COUNTRY'], axis=1).fillna(0)))

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 1494 and the array at index 1 has size 851

In [19]:

model = LinearRegression()
model.fit(df_X_train_fr, df_y_train_fr['TARGET'])
# model.predict(df_X_train_fr)
show_spearman_correlation(model.predict(df_X_train_fr))

0.22230679657142144


Linear regression with regularization

Linear regression with cross validation

Polynomial regression

Polynomial regression with regularization

Polynomial regression with cross validation

Neural networks

In [ ]:
y_predict = model.predict(df_X_train_fr)


0.22230679657142144


In [ ]:
from sklearn.linear_model import Ridge
model = Ridge(alpha=10)
model.fit(df_X_train_fr, df_y_train_fr['TARGET'])

Ridge(alpha=10)

In [ ]:
result_y = model.predict(df_X_train_fr)
print(spearmanr(result_y, df_y_train_fr['TARGET']).correlation)

0.23177915829632
